<a href="https://colab.research.google.com/github/saiteja-ms/Deepfake-detection-using-Deep-Learning-ResNet-and-LSTM-/blob/main/Model_and_train_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set hardware accelerator
#Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=0095255374986949f68a069620067141a10767c60f45428d034748938460f2cb
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import glob
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
# Check if the file is corrupted or not
def validate_video(vid_path, train_transforms):
    transform = train_transforms
    count = 20
    video_path = vid_path
    frames = []
    a = int(100/count)
    first_frame = np.random.randint(0,a)
    temp_video = video_path.split('/')[-1]
    for i, frame in enumerate(frame_extract(video_path)):
        frames.append(transform(frame))
        if(len(frames) == count):
            break
    frames = torch.stack(frames)
    frames = frames[:count]
    return frames
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            yield image

im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

video_fil = glob.glob('/content/drive/MyDrive/Face_only_data/*.mp4')
print("Total no of videos :",len(video_fil))
print(video_fil)
count = 0
for i in video_fil:
    try:
        count+=1
        validate_video(i,train_transforms)
    except:
        print("Number of video processed: ",count," Remaining : ",(len(video_fil) - count))
        print("Corrupted video is :", i)
        continue
print((len(video_fil) - count))

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-plk_lhfd/dlib_511fe1a5b5ce41869787b18bc47695fd/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
import json
import glob
import numpy as np
import cv2
import copy
import random
video_files =  glob.glob('/content/drive/MyDrive/Face_only_data/*.mp4')
random.shuffle(video_files)
random.shuffle(video_files)
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<100):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

In [ ]:
import json
import os
import pandas as pd

# Path to the JSON file
json_file_path = '/path/to/your/json_file.json'

# Directory where your extracted video files are stored
video_dir = '/path/to/your/extracted_videos'

# Output CSV file path
output_csv_path = '/path/to/your/output.csv'

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# List all video files in the video directory
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]

# Initialize a list to hold the CSV data
csv_data = []

# Iterate over the video files and check their labels
for video_file in video_files:
    if video_file in data:
        label = data[video_file]['label']
        video_path = os.path.join(video_dir, video_file)
        csv_data.append([video_path, label])
    else:
        print(f"Video file {video_file} not found in JSON data")

# Create a DataFrame from the CSV data
df = pd.DataFrame(csv_data, columns=['file', 'label'])

# Save the DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"CSV file saved to {output_csv_path}")


In [ ]:
# load the video name and labels from csv
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition

In [ ]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import cv2
import numpy as np
import torch
import os

header_list = ["file", "label"]
labels = pd.read_csv('', names=header_list)

# Assuming video_files is defined somewhere in your code
video_files = labels['file'].tolist()  # Ensure video_files is correctly populated

train_videos = video_files[:int(0.8 * len(video_files))]
valid_videos = video_files[int(0.8 * len(video_files)):]
print("train : ", len(train_videos))
print("test : ", len(valid_videos))

# Define the function number_of_real_and_fake_videos
def number_of_real_and_fake_videos(data_list):
    header_list = ["file", "label"]
    labels = pd.read_csv('', names=header_list)
    fake = 0
    real = 0
    for i in data_list:
        temp_video = i.split('/')[-1]
        label_rows = labels.loc[labels["file"] == temp_video]
        if not label_rows.empty:
            label = label_rows.iloc[0, 1]
            if label == 'FAKE':
                fake += 1
            if label == 'REAL':
                real += 1
    return real, fake

print("TRAIN: ", "Real", number_of_real_and_fake_videos(train_videos)[0], " FAKE:", number_of_real_and_fake_videos(train_videos)[1])
print("TEST: ", "Real", number_of_real_and_fake_videos(valid_videos)[0], " FAKE:", number_of_real_and_fake_videos(valid_videos)[1])

im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

class video_dataset(Dataset):
    def __init__(self, video_names, labels, sequence_length=60, transform=None):
        self.video_names = video_names
        self.labels = labels
        self.transform = transform
        self.count = sequence_length

    def __len__(self):
        return len(self.video_names)

    def __getitem__(self, idx):
        video_path = self.video_names[idx]
        print(f"Processing video: {video_path}")

        #check if the video file exists
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file {video_path} not found")

        frames = []
        a = int(100 / self.count)
        first_frame = np.random.randint(0, a)
        temp_video = video_path.split('/')[-1]
        label_rows = self.labels.loc[self.labels['file'] == temp_video]
        if not label_rows.empty:
            label = label_rows.iloc[0, 1]
            if label == 'FAKE':
                label = 0
            elif label == 'REAL':
                label = 1
            else:
                raise ValueError("Unknown label value")
        else:
            raise ValueError(f"Label for video {video_path} not found")

        frame_count = 0
        for i, frame in enumerate(self.frame_extract(video_path)):
            frames.append(self.transform(frame))
            frame_count += 1
            if len(frames) == self.count:
                break

        if len(frames) == 0:
            print(f"No frames extracted for video: {video_path}")
            raise ValueError(f"No frames extracted for video: {video_path}")

        print(f"Extracted {frame_count} frames for video: {video_path}")

        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames, label

    def frame_extract(self, path):
        vidObj = cv2.VideoCapture(path)
        success = True
        while success:
            success, image = vidObj.read()
            if success and image is not None:
                yield image
            else:
                break

train_data = video_dataset(train_videos, labels, sequence_length=10, transform=train_transforms)
val_data = video_dataset(valid_videos, labels, sequence_length=10, transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
valid_loader = DataLoader(val_data, batch_size=4, shuffle=True, num_workers=2)

image, label = train_data[0]
# im_plot(image[0,:,:,:]) # Assuming im_plot is defined elsewhere


train :  18548
test :  4638
TRAIN:  Real 2999  FAKE: 15549
TEST:  Real 532  FAKE: 4106
Processing video: 000.mp4


FileNotFoundError: Video file 000.mp4 not found

In [ ]:
# Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim = 2048, lstm_layers = 1, hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) # Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional=bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, num_classes, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, -1)
        x_lstm, _ = self.lstm(x)
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim = 1)))

In [ ]:
# Instantiate the model
model = Model(2).cuda()

# Dummy input tensor
input_tensor = torch.from_numpy(np.empty((1, 20, 3, 112, 112))).type(torch.cuda.FloatTensor)

# Forward pass
fmap, output = model(input_tensor)
print(f"Feature Map Shape: {fmap.shape}")
print(f"Output Shape: {output.shape}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature Map Shape: torch.Size([20, 2048, 4, 4])
Output Shape: torch.Size([1, 2])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [ ]:
import torch
from torch.autograd import Variable
import time
import os
import sys

def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer):
    model.train()
    losses = AverageMeter()
    t = []
    for i, (inputs, targets) in enumerate(data_Loader):
        if torch.cuda.is_available():
            targets =  targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        _, outputs = model(inputs)
        loss = criterion(outputs, targets.type(torch.cuda.LongTensor))
        acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d / %d] [Loss: %f, Acc: %.2f%%]"
                    %(
                        epoch,
                        num_epochs,
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg))
    torch.save(model.state_dict(), '')
    return losses.avg, accuracies.avg

def test(epoch, model, data_loader, criterion):
    print('Testing')
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []
    count = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if torch.cuda.is_available():
                targets = targets.cuda().type(torch.cuda.FloatTensor)
                inputs = inputs.cuda()
            _, outputs = model(inputs)
            loss = torch.mean(criterion(outputs, targets.type(torch.cuda.LongTensor)))
            acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
            _,p = torch.max(outputs, 1)
            true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
            pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))
            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    %(
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                    )
            )
        print('\nAccuracy {}'.format(accuracies.avg))
    return true, pred, losses.avg, accuracies.avg

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum/self.count

def calculate_accuracy(outputs, targets):
    batch_size = target.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1,-1))
    n_correct_elems = correct.float().sum().item()
    return 100*n_correct_elems / batch_size


In [ ]:
import seaborn as sns
#Output confusion matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sns.set(font_scale = 1.4) # for label size
    sns.heatmap(df_cm, annot = True, annot_kws={"size":16})
    plt.ylabel('Actual label',size=20)
    plt.xlabel('Predicted label',size=20)
    plt.xticks(np.arange(2),['Fake','Real'],size=16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size=16)
    plt.ylim([2,0])
    plt.show()
    calc_acc = (cm[0][0] + cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
    print("Calculated Accuracy", calc_acc*100)

In [ ]:
def plot_loss(train_loss_avg, test_loss_avg, num_epochs):
    loss_train = train_loss_avg
    loss_val = test_loss_avg
    print(num_epochs)
    epochs = range(1, num_epochs+1)
    plt.plot(epochs, loss_train, 'g', label='Training_loss')
    plt.plot(epochs, loss_val, 'b', label='validation_loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_accuracy(train_accuracy, test_accuracy, num_epochs):
    loss_train = train_accuracy
    loss_val = test_accuracy
    epochs = range(1, num_epochs+1)
    plt.plot(epochs, loss_train, 'g', label='Training accuracy')
    plt.plot(epcohs, loss_val, 'b', label='Validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
#learning rate
lr = 1e-5
#number of epochs
num_epochs = 20

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

criterion = nn.CrossEntropyLoss().cuda()
train_loss_avg = []
train_accuracy = []
test_loss_avg = []
test_accuracy = []
for epoch in range(1, num_epochs+1):
    l, acc = train_epoch(epoch, num_epochs, train_loader, model, criterion, optimizer)
    train_loss_avg.append(l)
    train_accuracy.append(acc)
    true, pred, tl, t_acc = test(epoch, model, valid_loader, criterion)
    test_loss_avg.append(tl)
    test_accuracy.append(t_acc)
plot_loss(train_loss_avg, test_loss_avg, len(train_loss_avg))
plot_accuracy(train_accuracy, test_accuracy, len(train_accuracy))
print(confusion_matrix(true, pred))
print_confusion_matrix(true, pred)

NameError: name 'data_Loader' is not defined